In [1]:
from Bio.PDB import PDBParser, MMCIFParser
from Bio.SVDSuperimposer import SVDSuperimposer
import numpy as np

def load_structure(filename, struct_id="structure"):
    if filename.endswith(".pdb"):
        parser = PDBParser(QUIET=True)
    elif filename.endswith(".cif") or filename.endswith(".mmcif"):
        parser = MMCIFParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format: must be .pdb or .cif/.mmcif")
    structure = parser.get_structure(struct_id, filename)
    return structure

def get_ligand_coords(structure):
    lig_coords = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.id[0] != " ":
                    for atom in residue:
                        lig_coords.append(atom.get_coord())
    return lig_coords

def find_pocket_residues(structure, lig_coords, cutoff=4.0):
    pocket_res = set()
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.id[0] == " ":
                    for atom in residue:
                        for lig_coord in lig_coords:
                            d = np.linalg.norm(atom.get_coord() - lig_coord)
                            if d <= cutoff:
                                pocket_res.add( (chain.id, residue.id[1]) )
                                break
    return pocket_res

def get_ca_coords(structure):
    ca_coords = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    ca_coords.append(residue['CA'].get_coord())
    return ca_coords

def get_sidechain_coords(structure, res_id_set):
    sc_coords = {}
    for model in structure:
        for chain in model:
            for residue in chain:
                key = (chain.id, residue.id[1])
                if key in res_id_set and residue.id[0] == " ":
                    for atom in residue:
                        if atom.get_name() in ['N','CA','C','O','OXT'] or atom.get_name().startswith('H'):
                            continue
                        tag = (key, atom.get_name())
                        sc_coords[tag] = atom.get_coord()
    return sc_coords


def kabsch_rmsd(coords1, coords2):
    sup = SVDSuperimposer()
    sup.set(np.array(coords1), np.array(coords2))
    sup.run()
    return sup.get_rms()

def calculate_CA_rmsd(ref_fn, cmp_fn):
    ref_struct = load_structure(ref_fn)
    cmp_struct = load_structure(cmp_fn)
    ref_ca = get_ca_coords(ref_struct)
    cmp_ca = get_ca_coords(cmp_struct)
    return kabsch_rmsd(ref_ca, cmp_ca)
    
def calculate_sc_rmsd(ref_fn, cmp_fn, cutoff=4.0):
    ref_struct = load_structure(ref_fn)
    cmp_struct = load_structure(cmp_fn)
    lig_coords = get_ligand_coords(ref_struct)
    pocket_res = find_pocket_residues(ref_struct, lig_coords, cutoff=cutoff)
    ref_sc = get_sidechain_coords(ref_struct, pocket_res)
    cmp_sc = get_sidechain_coords(cmp_struct, pocket_res)
    tags = sorted(list(ref_sc.keys()))
    coords1 = [ref_sc[tag] for tag in tags if tag in cmp_sc]
    coords2 = [cmp_sc[tag] for tag in tags if tag in cmp_sc]
    if len(coords1) == 0 or len(coords2) == 0 or len(coords1) != len(coords2):
        raise ValueError("Sidechain coordinate extraction failed (no pocket residues or atoms)")
    return kabsch_rmsd(coords1, coords2)

In [2]:
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from functools import partial

def rmsd_worker(row, model, col='lmpnn'):
    lmpnn_path = row[(col, 'path')]
    model_path = row[(model, 'path')]
    try:
        ca = calculate_CA_rmsd(lmpnn_path, model_path)
        sc = calculate_sc_rmsd(lmpnn_path, model_path)
    except Exception as e:
        print(f"Error processing {lmpnn_path} and {model_path}: {e}")
        ca = np.nan
        sc = np.nan
    return (ca, sc)

def process(df, model, col='lmpnn', pool_size=cpu_count()):
    rows = [row for _, row in df.iterrows()]
    worker = partial(rmsd_worker, model=model, col=col)
    with Pool(pool_size) as pool:
        results = list(tqdm(pool.imap(worker, rows), total=len(rows)))
    ca_rmsd, sc_rmsd = zip(*results)
    df[(model, 'ca_rmsd')] = ca_rmsd
    df[(model, 'sc_rmsd')] = sc_rmsd
    return df

path = 'lmpnn_filt_data_added.parquet'
df = pd.read_parquet(path)

df = process(df, model='af3', col='lmpnn', pool_size=8)
df = process(df, model='boltz', col='lmpnn', pool_size=8)

100%|██████████| 36/36 [00:00<00:00, 56.09it/s]


In [3]:
df.to_parquet('rmsd_1st.parquet')
df

diffusion                           lmpnn             \
                      id     batch                   tag        ddg   
0    result_7_packed_3_1  pht_demo   result_7_packed_3_1 -27.749462   
1   result_61_packed_8_1  pht_demo  result_61_packed_8_1 -26.640675   
2    result_7_packed_8_1  pht_demo   result_7_packed_8_1 -25.789537   
3    result_7_packed_2_1  pht_demo   result_7_packed_2_1 -25.547806   
4   result_16_packed_4_1  pht_demo  result_16_packed_4_1 -25.318375   
5   result_16_packed_7_1  pht_demo  result_16_packed_7_1 -24.987370   
6    result_7_packed_4_1  pht_demo   result_7_packed_4_1 -23.823254   
7   result_29_packed_7_1  pht_demo  result_29_packed_7_1 -22.682110   
8   result_59_packed_1_1  pht_demo  result_59_packed_1_1 -22.405994   
9   result_59_packed_6_1  pht_demo  result_59_packed_6_1 -21.156240   
10   result_7_packed_1_1  pht_demo   result_7_packed_1_1 -21.103624   
11  result_24_packed_8_1  pht_demo  result_24_packed_8_1 -20.956982   
12  result_24_packed_7_1  pht_demo  result_24_packed_7_1 -20.956400   
13  result_59_packed_3_1  pht_demo  result_59_packed_3_1 -20.733452   
14  result_55_packed_1_1  pht_demo  result_55_packed_1_1 -20.355864   
15  result_59_packed_8_1  pht_demo  result_59_packed_8_1 -20.324280   
16  result_24_packed_3_1  pht_demo  result_24_packed_3_1 -20.292067   
17  result_24_packed_6_1  pht_demo  result_24_packed_6_1 -20.236935   
18  result_59_packed_7_1  pht_demo  result_59_packed_7_1 -20.159588   
19  result_55_packed_2_1  pht_demo  result_55_packed_2_1 -19.940771   
20  result_24_packed_4_1  pht_demo  result_24_packed_4_1 -19.784569   
21  result_24_packed_5_1  pht_demo  result_24_packed_5_1 -19.718063   
22  result_10_packed_4_1  pht_demo  result_10_packed_4_1 -19.292791   
23  result_29_packed_3_1  pht_demo  result_29_packed_3_1 -19.285507   
24  result_55_packed_4_1  pht_demo  result_55_packed_4_1 -19.126881   
25  result_24_packed_1_1  pht_demo  result_24_packed_1_1 -18.947884   
26  result_55_packed_5_1  pht_demo  result_55_packed_5_1 -18.944952   
27  result_24_packed_2_1  pht_demo  result_24_packed_2_1 -18.919436   
28  result_59_packed_2_1  pht_demo  result_59_packed_2_1 -18.819977   
29  result_59_packed_5_1  pht_demo  result_59_packed_5_1 -18.560551   
30  result_16_packed_2_1  pht_demo  result_16_packed_2_1 -18.512140   
31  result_55_packed_7_1  pht_demo  result_55_packed_7_1 -18.505692   
32   result_4_packed_3_1  pht_demo   result_4_packed_3_1 -18.452574   
33  result_15_packed_7_1  pht_demo  result_15_packed_7_1 -18.338469   
34  result_29_packed_6_1  pht_demo  result_29_packed_6_1 -18.066357   
35   result_5_packed_6_1  pht_demo   result_5_packed_6_1 -18.049738   

                                           \
        fa_rep res_totalscore  totalscore   
0    95.805969      -1.583578 -199.530838   
1   104.494186      -1.441779 -181.664185   
2    98.276131      -1.570810 -197.922104   
3   102.521637      -1.610180 -202.882706   
4    91.409203      -1.448796 -160.816406   
5    88.052048      -1.478883 -164.156021   
6   103.836670      -1.429071 -180.062973   
7    75.388725      -1.675448 -177.597519   
8    70.000290      -1.624849 -180.358276   
9    66.255737      -1.786232 -198.271744   
10   97.976486      -1.563029 -196.941681   
11   73.818649      -1.721328 -194.510040   
12   75.090805      -1.810861 -204.627274   
13   66.947762      -1.525246 -169.302292   
14   69.711243      -1.708151 -182.772202   
15   65.809105      -1.677221 -186.171524   
16   68.647247      -1.709947 -193.223969   
17   76.706245      -1.489178 -168.277100   
18   69.610352      -1.726851 -191.680466   
19   66.307579      -1.638223 -175.289825   
20   75.684486      -1.614937 -182.487900   
21   73.074364      -1.775147 -200.591599   
22   83.306686      -1.661632 -177.794617   
23   81.779648      -1.521618 -161.291504   
24   67.392075      -1.756452 -187.940399   
25   74.767509      -1.562683 -176.583221   
26   71.573624      -1.511618 -161.743149   
27   75.802